In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score, train_test_split
import matplotlib.pyplot as plt
import scipy.optimize as opt
%matplotlib inline

In [2]:
data = datasets.load_boston()

In [3]:
targets = pd.DataFrame(data['target'],columns=['targets'])

In [4]:
data = pd.DataFrame(data['data'], columns=data['feature_names'])

In [5]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
data_train = data[:378]
data_test = data[378:]
target_train =targets[:378]
target_test = targets[378:]

In [6]:
data_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Алгоритм:
Просмотрим все признаки. Посчитаем минимум в MSE в линейной регрессии. Делить в вершине будем по признаку, имеющему наименьшую ошибку. При этом каждый раз рассматриваем признаки, еще не фигурировавшие в разделении. Далее каждую вершину разделим по принципу: значение признака, по которому делим выборку, больше среднего значения этого признака в выборке или же меньше. Метод predict работает также на сравнении значения признака со средним. В качестве ответа выдается среднее значение из листа.

In [495]:
class Decision_tree():
    def __init__(self, max_depth = 3, split_l=[] , split_r = [], T = [], feach_for_div = [], answers_l = [], answers_r = []):
        self.max_depth = max_depth
        self.split_l = split_l
        self.split_r = split_r
        self.features = []
        self.T = T
        self.feach_for_div = feach_for_div
        self.answers_l = answers_l
        self.answers_r = answers_r
    def fit(self, data, target):
        self.features = data.columns
        res_x = []
        res_fun = []
        "MSE"
        def error(args,dat,target):
            err = (args[0]*dat + args[1] - target).T.dot(args[0]*dat + args[1] - target)
            return err.values[0][0]
        for depth in range(1,self.max_depth+1):
            if depth == 1:
                "Поиск признака с наименьшей ошибкой"
                for feach in self.features:
                    if feach not in self.feach_for_div[(len(self.feach_for_div)/len(self.features))*len(self.features):]:
                        dat = data[feach].reshape(data.shape[0],1)
                        optim = opt.minimize(fun = error, x0 = [1.0,1.0], method = 'Nelder-Mead', tol = 1e-6 , args = (dat,target))
                        res_x.append(optim.x)
                        res_fun.append(optim.fun)
                    else:
                        res_x.append((1,1))
                        res_fun.append(100000000)
                feach_for_split = res_fun.index(min(res_fun))
                self.T.append(data[self.features[feach_for_split]].mean())
                spl_r = []
                spl_l = []
                "Построение разбиений"
                for t in range(len(data[self.features[feach_for_split]])):
                    elem = data[self.features[feach_for_split]][data.index[t]]
                    if elem > self.T[-1]:
                        spl_r.append(data.index[t])
                    else:
                        spl_l.append(data.index[t])
                self.split_l.append(spl_l)
                self.answers_l.append(target.ix[spl_l].values.mean())
                self.answers_r.append(target.ix[spl_r].values.mean())
                self.split_r.append(spl_r)
                self.feach_for_div.append(self.features[feach_for_split])
            else:
                "рекурсивное обучение"
                for point in range(2**(depth-2)):
                    clf = Decision_tree(max_depth = 1, split_l = self.split_l, split_r = self.split_r, T = self.T, 
                                        feach_for_div=self.feach_for_div, answers_l = self.answers_l, answers_r = self.answers_r)
                    clf.fit(pd.DataFrame(data.ix[self.split_l[2**(depth-2) - 1 + point]].values, columns=data.columns, index = self.split_l[2**(depth-2) - 1 + point]),
                            pd.DataFrame(target.ix[self.split_l[2**(depth-2) - 1 + point]].values,columns=target.columns, index = self.split_l[2**(depth-2) - 1 + point]))
                    clf.fit(pd.DataFrame(data.ix[self.split_r[2**(depth-2) - 1 + point]].values, columns=data.columns, index = self.split_r[2**(depth-2) - 1 + point]),
                            pd.DataFrame(target.ix[self.split_r[2**(depth-2) - 1 + point]].values,columns=target.columns, index = self.split_r[2**(depth-2) - 1 + point]))
                
    def predict(self,data_test):
        def num(d):
            num = 0
            for k in range(len(d)):
                num = (num*2 + 1) + d[k] 
            return num
        import copy
        data_res = copy.deepcopy(data_test)
        ans_predicted = []
        for j in range(data_res.shape[0]):
            elem = data_res.values[j]
            direction = [] # 1 = направо, 0 = налево
            "Вычисление положения в дереве"
            for i in range(self.max_depth):
                if i == 0:
                    if elem[self.features.get_loc(self.feach_for_div[0])] > self.T[0]:
                        direction.append(1)
                    else:
                        direction.append(0)
                else:
                    if elem[self.features.get_loc(self.feach_for_div[num(direction)])] > self.T[num(direction)]:
                        direction.append(1)
                    else:
                        direction.append(0)
            "Решение для элемента"
            if direction[-1] == 0:
                ans_predicted.append(self.answers_l[(num(direction)-1)/2])
            else:
                ans_predicted.append(self.answers_r[(num(direction)-1)/2])
        return pd.DataFrame(ans_predicted, index=data_res.index)
    def error(self, target_predicted, target):
        err = (np.array(target_predicted.values - target.values)).T.dot(np.array(target_predicted.values - target.values))
        return err[0][0]

In [499]:
clf = Decision_tree(max_depth = 3,split_l=[],split_r=[], feach_for_div=[], T= [], answers_l=[], answers_r=[])
clf.fit(data_train, target_train)

In [504]:
result = clf.predict(data_test = data_test)

In [505]:
result.head(10)

,0
378,24.351515
379,17.290164
380,24.351515
381,24.351515
382,17.290164
383,17.290164
384,17.290164
385,17.290164
386,17.290164
387,17.290164


In [507]:
print "Ошибка на тестовых данных: ", clf.error(result, target_test)
print "Ошибка на обучающих данных: ", clf.error(clf.predict(data_train), target_train)
print "Средняя ошибка на тестовых данных: ", clf.error(result, target_test)/np.shape(target_test)[0]
print "Средняя ошибка на обучающих данных: ", clf.error(clf.predict(data_train), target_train)/np.shape(target_train)[0]

Ошибка на тестовых данных:  7250.74741447
Ошибка на обучающих данных:  15102.6632611
Средняя ошибка на тестовых данных:  56.6464641756
Средняя ошибка на обучающих данных:  39.9541356115
